# Plant species recommender system - iteration 1
This is the first iteration of the recommender system. Plant species that contribute positively to biodiversity are recommended based on the plant species the user already has in their garden. 
In this NoteBook, a user profile is used. When necessary, example inputs are printed. These can be used to run the system. However, other inputs may be used if desired, but they must match with plant species in the database (recommender_system.csv). Otherwise the system cannot work.

# 1. Recommend plant species similar to those present in the user's garden
## 1.2 Data cleaning

### 1.1 Import libraries

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image
import numpy as np
import pandas as pd

### 1.2 Import CSV file with plant data
This CSV was generated earlier as a result of web scraping and data cleaning in the .py files

In [37]:
# Create a dataframe from the CSV file with plant data
df_plants = pd.read_csv("recommender_system.csv", encoding="UTF-8")

### 1.3 Create user profile
The user profile is filled in using the information given by the user when using the application. When a user adds a plant species in their garden, it is added to this list. However it is only possible to add plants known by the application (i.e. present in the CSV file).

In [38]:
list_user = [ ] 
number_plants = int(input("How many plants do you already have in your garden? "+"\033[1m"+"(for example 3)"+"\033[0m"))
  
for i in range(0, number_plants): 
    print("Type the name of a plant that is in your garden. Note that the recommender system only works with plants that are in its database.")
    print("\033[1m"+"Examples are: "+"\033[0m")
    print("\033[1m"+"Plant 1: Pontische Rododendron Rhododendron ponticum Roseum"+"\033[0m")
    print("\033[1m"+"Plant 2: Blauwe regen Wisteria sinensis Boskoop"+"\033[0m")
    print("\033[1m"+"Plant 3: Franse lavendel Lavandula stoechas subsp pedunculata"+"\033[0m")
    input_user = input("Enter the name of one of your plants: ")
    list_user.append(input_user) 
      
print("The user has the following plants already in his garden: ", list_user)

How many plants do you already have in your garden? (for example 3)3
Type the name of a plant that is in your garden. Note that the recommender system only works with plants that are in its database.
Examples are: 
Plant 1: Pontische Rododendron Rhododendron ponticum Roseum
Plant 2: Blauwe regen Wisteria sinensis Boskoop
Plant 3: Franse lavendel Lavandula stoechas subsp pedunculata
Enter the name of one of your plants: Pontische Rododendron Rhododendron ponticum Roseum
Type the name of a plant that is in your garden. Note that the recommender system only works with plants that are in its database.
Examples are: 
Plant 1: Pontische Rododendron Rhododendron ponticum Roseum
Plant 2: Blauwe regen Wisteria sinensis Boskoop
Plant 3: Franse lavendel Lavandula stoechas subsp pedunculata
Enter the name of one of your plants: Blauwe regen Wisteria sinensis Boskoop
Type the name of a plant that is in your garden. Note that the recommender system only works with plants that are in its database.
Ex

### 1.4 Transform text to feature vectors
Transforming the text of all cells into vectors (numerical values) will allow to measure the similarities between the plant species already present in the user's garden with all the other plant species in the application's database (CSV file). To do this, all columns must be merged into one. Some columns like the Name column are not taken into account, because it is not part of the plant properties.

In [39]:
# Create a copy of df_plants to avoid unwanted errors
df_copy = df_plants.copy()
df_copy = df_copy.drop(["Name"], axis=1)
df_copy = df_copy.drop(["Unnamed: 0"], axis=1)
df_copy = df_copy.drop(["Photo"], axis=1)
df_copy = df_copy.drop(["Maintenance description"], axis=1)

# Create a new df with 1 column that contains the values of all columns
df_recom = pd.DataFrame()
df_recom["Recommendation"]= df_copy[df_copy.columns[1:]].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df_recom

,Recommendation
0,Borderplanten GeenVogels Bijen Vlinders GeenEg...
1,Borderplanten Vogels GeenBijen GeenVlinders Ge...
2,Borderplanten GeenVogels GeenBijen Vlinders Ge...
3,Borderplanten GeenVogels Bijen Vlinders GeenEg...
4,Borderplanten GeenVogels Bijen Vlinders GeenEg...
...,...
926,Heesters en Struiken Vogels GeenBijen GeenVlin...
927,Heesters en Struiken Vogels Bijen Vlinders Gee...
928,Borderplanten GeenVogels Bijen Vlinders GeenEg...
929,Heesters en Struiken Vogels Bijen Vlinders Gee...


In [40]:
# Add a column ID with an ID for each row
df_recom["ID"]=[i for i in range(0,df_recom.shape[0])]

# Merge the two df df_plants and df_recom to retrieve the plant species names
df_recom = pd.merge(df_plants, df_recom, on="ID")
df_recom = df_recom.drop(["Unnamed: 0"], axis=1)
df_recom

,ID,Name,Type,Birds,Bees,Butterflies,Hedgehogs,Squirrels,Flowering period,Leaf color,Flower color,Evergreen,Plant growth,Sun exposure,Soil moisture,Maintenance,Maintenance description,Photo,Recommendation
0,0,Aar ereprijs Veronica spicata,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Augustus,BladGroen,BloemWit,NietWintergroen,50.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels Bijen Vlinders GeenEg...
1,1,Adderwortel Persicaria bistorta,Borderplanten,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei Juni Juli,BladGroen,BloemRoze,NietWintergroen,70.0,Lichteschaduw Zon,Vochthoudend vochtig,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten Vogels GeenBijen GeenVlinders Ge...
2,2,Afrikaanse lelie Agapanthus,Borderplanten,GeenVogels,GeenBijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Augustus,BladGroen,BloemBlauw,NietWintergroen,100.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels GeenBijen Vlinders Ge...
3,3,Akelei Aquilegia viridiflora Chocolate Soldier,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemPaars BloemBruin,NietWintergroen,60.0,Schaduw,Vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig. Zaa...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels Bijen Vlinders GeenEg...
4,4,Alpenaster Aster alpinus Happy End,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemRoze,Wintergroen,20.0,Zon,Vochthoudend vochtig,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels Bijen Vlinders GeenEg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,926,Zuurbes Berberis x lologensis Apricot Queen,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemGeel BloemOranje,Wintergroen,125.0,Lichteschaduw Zon,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Heesters en Struiken Vogels GeenBijen GeenVlin...
927,927,Zwarte aalbes Ribes nigrum Titania,Heesters en Struiken,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,April Mei Juli,BladGroen,BloemWit,NietWintergroen,200.0,"Halfschaduw, Zon",Vochthoudend,Hoog,Na het planten worden de takken voor 1/3 inges...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Heesters en Struiken Vogels Bijen Vlinders Gee...
928,928,Zwarte ereprijs Veronicastrum virginicum Diana,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Augustus,BladGroen,BloemWit,NietWintergroen,150.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels Bijen Vlinders GeenEg...
929,929,Zwarte toorts Verbascum nigrum,Heesters en Struiken,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,September Augustus Juli Juni,BladGroen,BloemGeel,NietWintergroen,100.0,Zon,Droog,Laag,"De plant is groenblijvend, snoei is niet nodig...","https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Heesters en Struiken Vogels Bijen Vlinders Gee...


In [41]:
# Text to vector transformation using Term Frequency - Inverse Document Frequency vectorization
vecs=TfidfVectorizer().fit_transform(df_recom["Recommendation"].apply(lambda x: np.str_(x)))
vecs

<931x117 sparse matrix of type '<class 'numpy.float64'>'
	with 15870 stored elements in Compressed Sparse Row format>

### 1.5 Calculate the cosine similarity
The cosine similarity can be computed from the feature vectors to know the degree of similarity between two feature vectors.

In [42]:
sim=cosine_similarity(vecs) 

### 1.6 Get the list of plants similar to those in the user's garden

In [43]:
def recommend(user_plants):
    # Get the user's plant species ID
    id_plants=df_recom[df_recom["Name"]==user_plants]["ID"].values[0]
    
    # Calculates the similarity between user's plant species and all other plant species in the application's database
    scores=list(enumerate(sim[id_plants]))
    
    # Sort plant species in ascending order
    sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
    
    # Get the name of similar plant species from their ID
    plants=[df_recom[plants[0]==df_recom["ID"]]["Name"].values[0] for plants in sorted_scores]
    
    return plants

In [44]:
# Run the function for each plant species that the user has in his garden
for plant in list_user:
    rec_plants=recommend(plant)
# Remove the plant species that the user already has in his garden from the list of recommended plant species    
    for plant in list_user:
        if plant in list_user:
            rec_plants.remove(plant)

In [45]:
print('These are the 5 plant species that are most similar to the plant species the user already has in his garden: ',
      rec_plants[:5])

These are the 5 plant species that are most similar to the plant species the user already has in his garden:  ['Lavendel Lavandula', 'Lavendel Lavandula wit', 'Steenraket Erysimum linifolium Bowles Mauve', 'Fijnstraal Erigeron Wuppertal', 'Salie Salvia x sylvestris Mainacht']


# 2. Plant species recommendations
It is now clear which plant species are recommended to the user. The plant names should be linked to the corresponding information to properly inform the user.

## 2.1 Data cleaning
### 2.1.1 Create a ranking indicator
The order of the plant species is important since the first plant species on the list are the best recommendations. This is taken into account by creating the column "Ranking".

In [46]:
df_ranking = pd.DataFrame(rec_plants)
# Creation of a "Ranking" column indicating the ranking of the plant species
df_ranking['Ranking'] = df_ranking.reset_index().index
df_ranking.rename(columns = {0:"Name"}, inplace = True)
df_ranking

,Name,Ranking
0,Lavendel Lavandula,0
1,Lavendel Lavandula wit,1
2,Steenraket Erysimum linifolium Bowles Mauve,2
3,Fijnstraal Erigeron Wuppertal,3
4,Salie Salvia x sylvestris Mainacht,4
...,...,...
923,Trosvlier Sambucus racemosa,923
924,Trosvlier Sambucus racemosa Plumosa Aurea,924
925,Trosvlier Sambucus racemosa Sutherland Gold,925
926,Zuilbeuk Fagus sylvatica Dawyck,926


### 2.1.2 Create a complete dataframe


In [47]:
# Creation of a dataframe including the classification of plant species and all their properties
df_rec = pd.merge(df_ranking, df_plants, on="Name", how="inner")
df_rec = df_rec.drop(["Unnamed: 0"], axis=1)
df_rec

,Name,Ranking,ID,Type,Birds,Bees,Butterflies,Hedgehogs,Squirrels,Flowering period,Leaf color,Flower color,Evergreen,Plant growth,Sun exposure,Soil moisture,Maintenance,Maintenance description,Photo
0,Lavendel Lavandula,0,466,Borderplanten,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juni Juli,BladGrijs,BloemPaars BloemBlauw,Wintergroen,90.0,Zon,Droog,Gemiddeld,De plant hoeft niet echt gesnoeid te worden. O...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
1,Lavendel Lavandula wit,1,467,Borderplanten,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Juni Juli,BladGrijs,BloemWit,Wintergroen,60.0,Zon,Droog,Gemiddeld,De plant hoeft niet echt gesnoeid te worden. O...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
2,Steenraket Erysimum linifolium Bowles Mauve,2,735,Borderplanten,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,September Augustus Juli Juni Mei,BladGroen BladGrijs,BloemPaars,Wintergroen,60.0,Zon,Droog vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
3,Fijnstraal Erigeron Wuppertal,3,148,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Augustus Juli Juni,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
4,Salie Salvia x sylvestris Mainacht,4,660,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Juni Mei,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,Trosvlier Sambucus racemosa,923,784,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,April Mei,BladGroen,BloemGeel BloemGroen,NietWintergroen,300.0,Lichteschaduw Halfschaduw,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
924,Trosvlier Sambucus racemosa Plumosa Aurea,924,785,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,April Mei,BladGeel,BloemGeel BloemGroen,NietWintergroen,250.0,Lichteschaduw Halfschaduw,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
925,Trosvlier Sambucus racemosa Sutherland Gold,925,786,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei April,BladGroen BladGeel,BloemGeel BloemGroen,NietWintergroen,250.0,Lichteschaduw Halfschaduw,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
926,Zuilbeuk Fagus sylvatica Dawyck,926,923,Bomen,GeenVogels,GeenBijen,GeenVlinders,GeenEgels,Eekhoorns,Mei,BladGroen BladBruin,NaN,NietWintergroen,4000.0,NaN,NaN,NaN,NaN,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."


## 2.2. Results
Below are 5 plant species that are the best recommended plant species.

In [48]:
print('\033[1m'+'First recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[0]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[0]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[0]['Birds']+", "+df_rec.iloc[0]['Bees']+", "+df_rec.iloc[0]['Butterflies']+", "+df_rec.iloc[0]['Hedgehogs']+" and "+df_rec.iloc[0]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[0]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[0]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[0]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[0]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[0]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[0]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[0]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[0]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[0]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[0]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[0]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[0]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[0]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[0]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[0]['Photo'])

First recommended plant:
The name of the plant is Lavendel Lavandula
The type of the plant is Borderplanten
The plant attracts Vogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is Juni Juli
The leaf color of the plant is Grijs
The flower color of the plant is Paars Blauw
In the winter, the plant is Wintergroen
The maximum growth height is 90.0 cm
The required maintenance level is Gemiddeld
To properly maintain the plant, you must do the following: De plant hoeft niet echt gesnoeid te worden. Om de plant in model te houden kan men in het voorjaar en/of in de zomer de lange scheuten inkorten om de vorm van de plant te behouden.
This is a picture of the plant:


In [49]:
print('\033[1m'+'Second recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[1]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[1]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[1]['Birds']+", "+df_rec.iloc[1]['Bees']+", "+df_rec.iloc[1]['Butterflies']+", "+df_rec.iloc[1]['Hedgehogs']+" and "+df_rec.iloc[1]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[1]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[1]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[1]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[1]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[1]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[1]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[1]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[1]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[1]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[1]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[1]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[1]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[1]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[1]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[1]['Photo'])

Second recommended plant:
The name of the plant is Lavendel Lavandula wit
The type of the plant is Borderplanten
The plant attracts Vogels, GeenBijen, GeenVlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is Juni Juli
The leaf color of the plant is Grijs
The flower color of the plant is Wit
In the winter, the plant is Wintergroen
The maximum growth height is 60.0 cm
The required maintenance level is Gemiddeld
To properly maintain the plant, you must do the following: De plant hoeft niet echt gesnoeid te worden. Om de plant in model te houden kan men in het voorjaar en/of in de zomer de lange scheuten inkorten om de vorm van de plant te behouden.
This is a picture of the plant:


In [50]:
print('\033[1m'+'Third recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[2]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[2]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[2]['Birds']+", "+df_rec.iloc[2]['Bees']+", "+df_rec.iloc[2]['Butterflies']+", "+df_rec.iloc[2]['Hedgehogs']+" and "+df_rec.iloc[2]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[2]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[2]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[2]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[2]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[2]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[2]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[2]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[2]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[2]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[2]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[2]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[2]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[2]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[2]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[2]['Photo'])

Third recommended plant:
The name of the plant is Steenraket Erysimum linifolium Bowles Mauve
The type of the plant is Borderplanten
The plant attracts Vogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is September Augustus Juli Juni Mei
The leaf color of the plant is Groen Grijs
The flower color of the plant is Paars
In the winter, the plant is Wintergroen
The maximum growth height is 60.0 cm
The required maintenance level is Geen
To properly maintain the plant, you must do the following: De plant heeft geen bijzondere zorg nodig.
This is a picture of the plant:


In [51]:
print('\033[1m'+'Fourth recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[3]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[3]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[3]['Birds']+", "+df_rec.iloc[3]['Bees']+", "+df_rec.iloc[3]['Butterflies']+", "+df_rec.iloc[3]['Hedgehogs']+" and "+df_rec.iloc[3]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[3]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[3]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[3]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[3]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[3]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[3]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[3]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[3]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[3]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[3]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[3]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[3]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[3]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[3]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[3]['Photo'])

Fourth recommended plant:
The name of the plant is Fijnstraal Erigeron Wuppertal
The type of the plant is Borderplanten
The plant attracts GeenVogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is Augustus Juli Juni
The leaf color of the plant is Groen
The flower color of the plant is Paars Blauw
In the winter, the plant is NietWintergroen
The maximum growth height is 60.0 cm
The required maintenance level is Laag
To properly maintain the plant, you must do the following: De plant sterft elke winter af en loopt in het voorjaar weer uit. Net voor het uitlopen kan het oude loof afgeknipt of weggehaald worden.
This is a picture of the plant:


In [52]:
print('\033[1m'+'Fifth recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[4]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[4]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[4]['Birds']+", "+df_rec.iloc[4]['Bees']+", "+df_rec.iloc[4]['Butterflies']+", "+df_rec.iloc[4]['Hedgehogs']+" and "+df_rec.iloc[4]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[4]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[4]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[4]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[4]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[4]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[4]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[4]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[4]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[4]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[4]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[4]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[4]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[4]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[4]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[4]['Photo'])

Fifth recommended plant:
The name of the plant is Salie Salvia x sylvestris Mainacht
The type of the plant is Borderplanten
The plant attracts GeenVogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is Juli Juni Mei
The leaf color of the plant is Groen
The flower color of the plant is Paars Blauw
In the winter, the plant is NietWintergroen
The maximum growth height is 60.0 cm
The required maintenance level is Laag
To properly maintain the plant, you must do the following: De plant sterft elke winter af en loopt in het voorjaar weer uit. Net voor het uitlopen kan het oude loof afgeknipt of weggehaald worden.
This is a picture of the plant:


# References
VibhavSurve09 (2021) Recommendation System Cosine Similarity (Version 1.0) [Source code]. https://github.com/VibhavSurve09/recommendation-system-cosine-similarity